In [1]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import tqdm

In [2]:
import numpy as np

In [5]:
X, y = make_classification(
    n_samples=1000,
    n_features=50,
    n_informative=40,
    n_classes=10,
    random_state=0,
)

In [6]:
import torch

In [7]:
X = torch.from_numpy(X)
y = torch.from_numpy(y)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [12]:
class RandomForest:
    def __init__(self, num_feature, num_class, n_estimators=10, max_features=None,
                 min_samples_split=2, min_impurity_decrease=0, max_depth=np.inf):
        self.num_feature = num_feature
        self.num_class = num_class
        self.n_estimators = n_estimators
        self.min_samples_split = min_samples_split
        self.min_impurity_decrease = min_impurity_decrease
        self.max_depth = max_depth
        
        if not max_features:
            self.max_features = int(np.sqrt(self.num_feature))
        else:
            self.max_features = max_features
            
        self.trees = []
        for _ in range(self.n_estimators):
            self.trees.append(
                DecisionTreeClassifier(
                    self.max_features, num_class, min_samples_split, min_impurity_decrease, max_depth
                )
            )
    
    def fit(self, X, y):  
        N = len(X)
        for i in tqdm.tqdm(range(self.n_estimators)):
            idx = np.random.choice(range(N), size=N//2)
            feature_idx = np.random.choice(range(self.num_feature), size=self.max_features)
            self.trees[i].feature_idx = feature_idx
            
            X_subset = X[idx][:, feature_idx]
            y_subset = y[idx]
            self.trees[i].fit(X_subset, y_subset)
    
    def predict(self, X):
        y_preds = []
        for tree in self.trees:
            idx = tree.feature_idx
            y_preds.append(tree.predict(X[:, idx]))
        
        y_preds = torch.stack(y_preds).T
        out = []
        for p in y_preds:
            p = torch.bincount(p, minlength=self.num_class)
            out.append(p)
        out = torch.stack(out)
        return out.argmax(dim=1)

In [13]:
from sklearn.metrics import accuracy_score

In [18]:
clf = RandomForest(50, 10, n_estimators=80, max_features=40)
clf.fit(X_train, y_train)
rf_pred = clf.predict(X_test)
accuracy_score(y_test, rf_pred)

100%|███████████████████████████████████████████| 80/80 [00:14<00:00,  5.69it/s]


0.3